# Introduction

## CSI4106 - Introduction to Artificial Intelligence
## Project One 

Submitted On Behalf Of
Aiden Stevenson Bradwell | abrad060@uottawa.ca | 300064655

Chethin Manage | cman072@uottawa.ca | 300066367

**Dataset Selected**
- [CS:GO Round Winner Classification](https://www.kaggle.com/datasets/christianlillelund/csgo-round-winner-classification)


---

# Background

Counter Strike, Global Offensive is a first-person videogame, where two teams compete to either attack a target or to defend the target. 

Each game consists of two teams
Each game consists of 30 rounds, where the first to win 16 games (or both 15 in a tie situation) wins the game.
Each team consists of 5 players, and each player gets one life a round. 

The round begins with a 30 second purchase-period, where players can spend their money on different armor and weapons, to compete in the coming round. If the player survived the previous round, they keep their previous weapons and armor.

This dataset was collected during a professional tournament, where all players involved are competing on a high level of play. This assists in the accuracy of our network, as we can assume that the players involved are of a similar ability. Therefore, we can assume that each round is can be predicted based on the weapons and status of the team. 

Using the data of team health, players alive, weapons purchased, map of play, and additional game stats we seek to predict which team will win the current round. 

## Add Dataset and Joblibs into Runtime

In [ ]:
! pip install kaggle

%env KAGGLE_USERNAME=cmanage1
%env KAGGLE_KEY=b4a5756ebbc30f138ad5296cf61ba80a

! kaggle datasets download -d christianlillelund/csgo-round-winner-classification
! unzip csgo-round-winner-classification.zip 

! git clone https://github.com/cmanage1/ProjectOne-4106.git

## Clean Data

Data is using Professional Matches. Professional match rules state
- 20 second buy times 
- 1m 55s rounds (115s)

Since the data fromt he start of the round (where both teams have all players, and all weapons) would be 50/50 chances, we are only interested in predicting the winner based on the mid-game stats. This will be more representative of how the round is actaully going. 
- Filtering to snapshots only taken 60s to 120s mark so we're not considering end-round stats or buy-round factors


Additionally, within CSGO there is consideration needed for the map being played. It is knoweldge in the playerbase that some maps provide a firm advantage to one team. We have converted this attribute to a one-hot system to be used as inputs for the Multilayer processs and naive bayes

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

data = pd.read_csv('csgo_round_snapshots.csv')
data.head()

data = data[data['time_left'].between(60, 120.00)]

# Bucket gun data
data["ct_rifles"] = data["ct_weapon_ak47"] + data["ct_weapon_aug"] + data["ct_weapon_awp"] + data["ct_weapon_famas"] + data["ct_weapon_g3sg1"] + data["ct_weapon_famas"] + data["ct_weapon_galilar"] +  data["ct_weapon_m4a1s"] + data["ct_weapon_m4a4"] + data["ct_weapon_scar20"] + data["ct_weapon_sg553"] + data["ct_weapon_ssg08"] + data["ct_weapon_sg553"]
data["ct_heavy"] = data["ct_weapon_m249"] + data["ct_weapon_mag7"] + data["ct_weapon_negev"] + data["ct_weapon_nova"] + data["ct_weapon_sawedoff"] + data["ct_weapon_xm1014"]
data["ct_smg"] = data["ct_weapon_mac10"] + data["ct_weapon_bizon"] + data["ct_weapon_mp5sd"] + data["ct_weapon_mp7"] + data["ct_weapon_mp9"] + data["ct_weapon_p90"] + data["ct_weapon_ump45"]
data["ct_pistol"] = data["ct_weapon_cz75auto"] + data["ct_weapon_elite"] + data["ct_weapon_glock"] + data["ct_weapon_r8revolver"] + data["ct_weapon_deagle"] + data["ct_weapon_fiveseven"] + data["ct_weapon_usps"] + data["ct_weapon_p250"] + data["ct_weapon_p2000"] + data["ct_weapon_tec9"]

data["t_rifles"] = data["t_weapon_ak47"] + data["t_weapon_aug"] + data["t_weapon_awp"] + data["t_weapon_famas"] + data["t_weapon_g3sg1"] + data["t_weapon_famas"] + data["t_weapon_galilar"] + data["t_weapon_m4a1s"] + data["t_weapon_m4a4"] + data["t_weapon_scar20"] + data["t_weapon_sg553"] + data["t_weapon_ssg08"] + data["t_weapon_sg553"]
data["t_heavy"] = data["t_weapon_m249"] + data["t_weapon_mag7"] + data["t_weapon_negev"] + data["t_weapon_nova"] + data["t_weapon_sawedoff"] + data["t_weapon_xm1014"]
data["t_smg"] = data["t_weapon_mac10"] + data["t_weapon_bizon"] + data["t_weapon_mp5sd"] + data["t_weapon_mp7"] + data["t_weapon_mp9"] + data["t_weapon_p90"] + data["t_weapon_ump45"]
data["t_pistol"] = data["t_weapon_cz75auto"] + data["t_weapon_elite"] + data["t_weapon_glock"] + data["t_weapon_r8revolver"] + data["t_weapon_deagle"] + data["t_weapon_fiveseven"] + data["t_weapon_usps"] + data["t_weapon_p250"] + data["t_weapon_p2000"] + data["t_weapon_tec9"]

data["ct_grenades"] = data["ct_grenade_hegrenade"] + data["ct_grenade_flashbang"] + data["ct_grenade_smokegrenade"] + data["ct_grenade_incendiarygrenade"] + data["ct_grenade_molotovgrenade"] + data["ct_grenade_decoygrenade"]
data["t_grenades"] = data["t_grenade_hegrenade"] + data["t_grenade_flashbang"] + data["t_grenade_smokegrenade"] + data["t_grenade_incendiarygrenade"] + data["t_grenade_molotovgrenade"] + data["t_grenade_decoygrenade"]

y = data["round_winner"]

# Create labels for dataframe 
labels = ["time_left", "t_score", "ct_score", 
          "bomb_planted", "ct_armor", "t_armor", "ct_helmets", 
          "t_helmets", "ct_defuse_kits", "ct_players_alive", 
          "t_players_alive", "ct_rifles", "ct_heavy", "ct_smg", 
          "ct_pistol", "ct_grenades", "t_rifles", "t_heavy", 
          "t_smg", "t_pistol", "t_grenades", "de_inferno" ,"de_dust2", 
          "de_nuke", "de_mirage", "de_overpass"]


# One-Hot-Conversion for training which requires numerical data opposed to string classification. 
one_hot_data = data.copy()
one_hot_data["de_inferno"] = one_hot_data["map"] == "de_inferno"
one_hot_data["de_dust2"] = one_hot_data["map"] == "de_dust2"
one_hot_data["de_nuke"] = one_hot_data["map"] == "de_nuke"
one_hot_data["de_mirage"] = one_hot_data["map"] == "de_mirage"
one_hot_data["de_overpass"] = one_hot_data["map"] == "de_overpass"

one_hot_data["de_inferno"] = one_hot_data["de_inferno"]*1
one_hot_data["de_dust2"] = one_hot_data["de_inferno"]*1
one_hot_data["de_nuke"] = one_hot_data["de_nuke"]*1
one_hot_data["de_mirage"] = one_hot_data["de_mirage"]*1
one_hot_data["de_overpass"] = one_hot_data["de_overpass"]*1

# Label data
one_hot_data = one_hot_data[labels]

# Naive Bayes

### Version One: Gaussian Naive Bayes<br>
  - Training / Testing Data split: 0.33
  - var_smoothing=1e-9
  - Accuracy: 0.6484263715832246 average over 25 k-fold scoring
  - Percision: 0.6582439299830605
  - Recall: 0.65372589890568

Note: changing splitting variable produces minute differences in prediction accurracy, in the range of 0.02 

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB

from sklearn.metrics import PrecisionRecallDisplay
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

X_train_naive, X_test_naive, y_train_naive, y_test_naive = train_test_split(one_hot_data, y, test_size=0.3)


gaussian_naive_byers = GaussianNB(var_smoothing=1e-9)
y_pred = gaussian_naive_byers.fit(X_train_naive, y_train_naive).predict(X_test_naive)

scores = cross_val_score(gaussian_naive_byers, one_hot_data, y, cv=25)
avg_score = scores.sum()/25
print(avg_score)

precision_score(y_test_naive, y_pred, average='macro')
recall_score(y_test_naive, y_pred, average='macro')


0.6780312884643674


0.6768089739093179

### Version Two: Multinomial Naive Bayes
  - Training Data split: 0.33 
  - ALPHA | | AVERAGE ACCURACY   |          PERCISION |         RECALL
    - 0.2 | |  0.635572548843207 | 0.6366622165467068 | 0.6354164031630551
    - 0.3 | |  0.635572548843207 | 0.6366622165467068 | 0.6354164031630551
    - 0.4 | |  0.635572548843207 | 0.6366622165467068 | 0.6354164031630551
    - 0.5 | |  0.635572548843207 | 0.6366622165467068 | 0.6354164031630551
    - 0.6 | |  0.635572548843207 | 0.6366622165467068 | 0.6354164031630551
    - 0.7 | |  0.635572548843207 | 0.6366622165467068 | 0.6354164031630551
    - 0.8 | |  0.635572548843207 | 0.6366622165467068 | 0.6354164031630551
    - 0.9 | |  0.635572548843207 | 0.6366622165467068 | 0.6354164031630551
    - 1.0 | |  0.635572548843207 | 0.6366622165467068 | 0.6354164031630551

Note: Changing the alpha does not seem to have an impact on the calculated results. 

In [ ]:
from sklearn.naive_bayes import MultinomialNB

multinomial_naive_bayes = MultinomialNB(alpha=0.1)
y_pred = multinomial_naive_bayes.fit(X_train_naive, y_train_naive).predict(X_test_naive)

scores = cross_val_score(multinomial_naive_bayes, one_hot_data, y, cv=25)
avg_score = scores.sum()/25

print(scores)
print(avg_score)

precision_score(y_test_naive, y_pred, average='macro')
recall_score(y_test_naive, y_pred, average='macro')

[0.67162746 0.65866539 0.67930869 0.66586654 0.66778685 0.68987038
 0.65914546 0.65578493 0.68362938 0.66298608 0.67450792 0.662506
 0.67642823 0.67402784 0.63736792 0.67627281 0.65129683 0.64553314
 0.66954851 0.67291066 0.65273775 0.64793468 0.67675312 0.66186359
 0.6493756 ]
0.6649494305255987


0.6675594341138347


### Version Three: Complement naive Bayes
  - Training Data /training data split: 0.33
  
  - Given (alpha, norm) -> (avg_score, percision, recall)
    - (1.0, False) -> (0.6345664462537298, 0.6418988648090815, 0.6418988648090815)
    - (0.5, False) -> (0.6345664462537298, 0.6418988648090815, 0.6418988648090815)
    - (0.01, False) -> (0.6345664462537298, 0.6418988648090815, 0.6418988648090815)
    - (1.0, True) -> (0.6165320198521584, 0.6068111455108359, 0.6068111455108359)
    - (0.5, True) -> (0.6146747034921206, 0.6070691434468525,	0.6070691434468525)
    - (0.01, True) -> (0.6134370923485224, 0.6060371517027864, 0.6060371517027864)


Note: A change in the average is detected when norm=True, however this method remains less accurate than its non-normalized compliment. Could the network be overfitting on non-normalized modes?


In [ ]:
from sklearn.naive_bayes import ComplementNB

complement_naive_bayes = ComplementNB(alpha=0.01, norm=False)
y_pred = complement_naive_bayes.fit(X_train_naive, y_train_naive).predict(X_test_naive)

scores = cross_val_score(complement_naive_bayes, one_hot_data, y, cv=25)
avg_score = scores.sum()/25

print(scores)
print(avg_score)

precision_score(y_test_naive, y_pred, average='macro')
recall_score(y_test_naive, y_pred, average='macro')

[0.67114738 0.65914546 0.67882861 0.66394623 0.66922708 0.68843015
 0.65962554 0.65530485 0.6831493  0.662506   0.674988   0.66154585
 0.67546807 0.67354777 0.6364073  0.67483189 0.65273775 0.6493756
 0.67002882 0.67291066 0.65225744 0.64793468 0.67723343 0.66090298
 0.64793468]
0.6647766213199299


0.6667829538461861

# Logistic Regression

[SCI-KIT Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

### Version One: Logistic Regression (LBFGS Solver) <br>
  - Training / Testing Data split: 0.33
  - Solver: lbfgs (Limited-memory Broyden–Fletcher–Goldfarb–Shanno Algorithm)
  - max_iter =1000
  - Accuracy: 0.6806017925736235 
  - Accuracy average over 25 k-fold: 0.6794047840521296
  - Percision: 0.6795798857368007
  - Recall: 0.6794047840521296





In [ ]:
from sklearn.linear_model import LogisticRegression

x_train_logistic, x_test_logistic, y_train_logistic, y_test_logistic = train_test_split(one_hot_data, y, test_size=0.3)
logistic_regr_1 = LogisticRegression(solver="lbfgs", max_iter=1000)

# Fitting the model to learn the relationship between x and y
logistic_regr_1.fit(x_train_logistic, y_train_logistic)

logistic_regr_1.predict(x_test_logistic)

# Check accuracy of prediction once
accuracy = logistic_regr_1.score(x_test_logistic, y_test_logistic)
print(accuracy)

# Check accuracy over 25 k-fold
scores = cross_val_score(logistic_regr_1, one_hot_data, y, cv=25)
avg_score = scores.sum()/25
print(avg_score)

# Check precision and recall 
print(precision_score(y_test_naive, y_pred, average='macro'))
print(recall_score(y_test_naive, y_pred, average='macro'))

### Version Two: Logistic Regression (Newton Solver)
  - Training / Testing Data split: 
  - Solver: newton-cg (Newton's method)
  - max_iter: 1000
  - Accuracy: 0.6903329065300896
  - Accuracy average over 25 k-fold: 0.6858476399451576
  - Percision: 0.6795798857368007
  - Recall: 0.6794047840521296
  
**Note:** This model yields the highest accuracy and accuracy average

In [ ]:
from sklearn.linear_model import LogisticRegression

x_train_logistic, x_test_logistic, y_train_logistic, y_test_logistic = train_test_split(one_hot_data, y, test_size=0.3)
logistic_regr_2 = LogisticRegression(solver="newton-cg", max_iter=1000)

# Fitting the model to learn the relationship between x and y
logistic_regr_2.fit(x_train_logistic, y_train_logistic)

logistic_regr_2.predict(x_test_logistic)

# Check accuracy of prediction once
accuracy = logistic_regr_2.score(x_test_logistic, y_test_logistic)
print(accuracy)

# Check accuracy over 25 k-fold
scores = cross_val_score(logistic_regr_2, one_hot_data, y, cv=25)
avg_score = scores.sum()/25
print(avg_score)

# Check precision and recall 
print(precision_score(y_test_naive, y_pred, average='macro'))
print(recall_score(y_test_naive, y_pred, average='macro'))

0.6854673495518566
0.6858476399451576
0.6678837336821521
0.6667829538461861


# Multi Layer Perceptron

## Version One: Default parameters
25-fold cross verification accuracy: 0.6690220037511477

[0.64954393, 0.65194431, 0.69899184, 0.64858377, 0.68795007, 0.67882861,

 0.70427268, 0.65482477, 0.68987038, 0.67018723, 0.674988,   0.68218915,

 0.66922708, 0.64330293, 0.66138329, 0.69980788, 0.65898175, 0.65754083,

 0.65898175, 0.68443804, 0.65946206, 0.65225744, 0.6820365,  0.63976945,

 0.66618636]


---

 ## Version Two: Warm start
 For this network, the warm start feature was enabled with the other parameters remaining default. 

25-fold cross verification accuracy: 0.6627808391705784


[0.66442631, 0.69131061, 0.62457993, 0.64378301, 0.69899184, 0.63850216,

 0.63610178, 0.67594815, 0.69131061, 0.63322132, 0.65338454, 0.68074892,

 0.668747,   0.656265,   0.66522574, 0.6820365,  0.65177714, 0.67002882,

 0.6426513,  0.6685879,  0.67291066, 0.64121037, 0.68443804, 0.6753122,

 0.65802113]



--- 


 ## Version Three: Small Hidden Layers
 For this network, the size of the hidden layers is reduced to 40 from the default 100. 

25-fold cross verification accuracy: 0.6730363590162898

[0.65050408, 0.65914546, 0.69755161, 0.66826692, 0.65914546, 0.67066731,

 0.67546807, 0.66490639, 0.69707153, 0.64906385, 0.70475276, 0.69803169,

 0.69323092, 0.66970715, 0.66666667, 0.6753122,  0.67675312, 0.66474544,

 0.69884726, 0.67867435, 0.6493756,  0.68876081, 0.68731988, 0.63928915,

 0.6426513 ]


---


## Version Four: Warm Start & Small Hidden Layers
 For this network, the size of the hidden layers is reduced to 40 from the default 100. Additionally, warm start is also applied

25-fold cross verification accuracy: 0.6791252179599456

[0.66442631, 0.69467115, 0.66298608, 0.66922708, 0.6831493,  0.70283245,

 0.71531445, 0.66298608, 0.66490639, 0.67882861, 0.68843015, 0.69227076,

 0.67786846, 0.67450792, 0.67002882, 0.6882805,  0.67098943, 0.65754083,

 0.68780019, 0.69692603, 0.69068204, 0.68491835, 0.65946206, 0.66762728,

 0.67146974]
0.

 ---

 ## Version Five: sgd solver
For this network, default parameters were restored with the solver being set to sgd (stochastic gradient descent)
25-fold cross verification accuracy: 0.6802963840208669

[0.65674508, 0.7018723,  0.7018723,  0.66346615, 0.69419107, 0.68602976,

 0.69035046, 0.66442631, 0.68698992, 0.68506961, 0.69323092, 0.68602976,

 0.68362938, 0.67930869, 0.6556196,  0.69740634, 0.66618636, 0.65850144,

 0.68683958, 0.68491835, 0.68731988, 0.67339097, 0.70509126, 0.66042267,

 0.65850144]


In [ ]:
from joblib import dump, load

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.33)
X_train_naive, X_test_naive, y_train_naive, y_test_naive = train_test_split(one_hot_data, y, test_size=0.3)

clf = MLPClassifier( 
                    max_iter=200,
                    hidden_layer_sizes=40,
                    activation='relu',
                    solver='adam',
                    alpha=0.0001,
                    batch_size='auto',
                    learning_rate='constant',
                    learning_rate_init=0.001,
                    power_t=0.5,
                    shuffle=True,
                    random_state=None,
                    tol=1e-4,
                    verbose=False,
                    warm_start=True,
                    momentum=0.9,
                    early_stopping=False,
                    validation_fraction=0.1,
                    beta_1=0.9,
                    beta_2=0.999,
                    epsilon=1e-8,
                    n_iter_no_change=10,
                    max_fun=15000)

clf.fit(X_train_naive.values, y_train_naive)
dump(clf, 'network_four_warm_start_small_hidden.joblib') 

clf.predict(X_test_naive.values)

# Classifier is used for classification 
scores = cross_val_score(clf, one_hot_data.values, y, cv=25)
avg_score = scores.sum()/25

print(scores)
print(avg_score)


[0.65962554 0.68843015 0.67690831 0.64330293 0.63370139 0.69755161
 0.64618339 0.67642823 0.6831493  0.67834854 0.69275084 0.68410946
 0.69995199 0.66538646 0.67243036 0.68491835 0.67771374 0.66762728
 0.69692603 0.69020173 0.68924111 0.66714697 0.6556196  0.66330451
 0.67002882]
0.6744394653576848


# Cross validation testing on the model

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from joblib import dump, load

# Already tested once within the model sections
# Testing here will be the second testing with modified params 

default_network = load('ProjectOne-4106/network_one_default.joblib') 
warm_start_network = load('ProjectOne-4106/network_two_warm_start.joblib') 
warm_start_small_hidden_layer_network = load('ProjectOne-4106/network_four_warm_start_small_hidden.joblib') 
small_hidden_layers_network = load('ProjectOne-4106/network_three_small_hidden_layers.joblib') 
lbfgs_network = load('ProjectOne-4106/network_five_sgd.joblib') 


scores = cross_val_score(default_network, one_hot_data.values, y, cv=25)
avg_score = scores.sum()/25
print("Default network || Accuracy Scores for Cross val (25 k-fold)")
print(avg_score)

scores = cross_val_score(warm_start_network, one_hot_data.values, y, cv=25)
avg_score = scores.sum()/25
print("Warm Start network || Accuracy Scores for Cross val (25 k-fold)")
print(avg_score)

scores = cross_val_score(small_hidden_layers_network, one_hot_data.values, y, cv=25)
avg_score = scores.sum()/25
print("Hidden Layer Small network || Accuracy Scores for Cross val (25 k-fold)")
print(avg_score)

scores = cross_val_score(warm_start_small_hidden_layer_network, one_hot_data.values, y, cv=25)
avg_score = scores.sum()/25
print("Warm start, small hidden layers network || Accuracy Scores for Cross val (25 k-fold)")
print(avg_score)

scores = cross_val_score(lbfgs_network, one_hot_data.values, y, cv=25)
avg_score = scores.sum()/25
print("lbfgs network || Accuracy Scores for Cross val (25 k-fold)")
print(avg_score)



Default network || Accuracy Scores for Cross val (25 k-fold)
0.6702311977985641
Warm Start network || Accuracy Scores for Cross val (25 k-fold)
0.6720381589584592
Hidden Layer Small network || Accuracy Scores for Cross val (25 k-fold)
0.676954588238441
Warm start, small hidden layers network || Accuracy Scores for Cross val (25 k-fold)
0.6792217498315581
lbfgs network || Accuracy Scores for Cross val (25 k-fold)
0.6790861661785195
